In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
from my_utils.preprocess import DataPreprocess
from my_utils.nfold_test import NFoldModel
from my_utils.dataloader import Workout_dataset
from my_utils.model import CNN_RNN_Resnet

In [2]:
# seed 고정
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(seed)
np.random.seed(seed)
rn.seed(seed)

In [3]:
features_path = './data/train_features.csv'
labels_path = './data/train_labels.csv'

preprocess = DataPreprocess(
    folds=10,seed=seed,acc_norm=10,gy_norm=2000,
    features_path=features_path,labels_path=labels_path)

In [4]:
preprocess.labels_to_csv()
preprocess.features_to_npy()

In [5]:
nfold_model = NFoldModel(batch_size=64, valid_ratio=4, lr=0.001, early_stop_patience=30, seed=42)
nfold_model.Model_class = CNN_RNN_Resnet
nfold_model.Workout_dataset = Workout_dataset
nfold_model.model_parameter = dict(leakyrelu_alpha = 0.1, 
    input_kernels = 20, input_kernel_width = 3, input_regularize_coeff=0.001,
    res_kernels = 60, res_kernel_width = 3, res_regularize_coeff=0.1, res_num = 5
    )
nfold_model.file_base_name = 'CNN_RNN_Resnet'

In [6]:
fold_list = [1,3,5,7,9]

In [7]:
nfold_model.nfold_train(fold_list)

o ./save/CNN_RNN_Resnet_fold_5.hdf5
Epoch 98/2000
35/35 [==============================] - 16s 445ms/step - loss: 0.6459 - accuracy: 0.9509 - val_loss: 0.9239 - val_accuracy: 0.8633

Epoch 00098: val_loss did not improve from 0.88250
Epoch 99/2000
35/35 [==============================] - 15s 439ms/step - loss: 0.6541 - accuracy: 0.9473 - val_loss: 0.9163 - val_accuracy: 0.8672

Epoch 00099: val_loss did not improve from 0.88250
Epoch 100/2000
35/35 [==============================] - 15s 436ms/step - loss: 0.6173 - accuracy: 0.9576 - val_loss: 0.9244 - val_accuracy: 0.8633

Epoch 00100: val_loss did not improve from 0.88250
Epoch 101/2000
35/35 [==============================] - 16s 444ms/step - loss: 0.6212 - accuracy: 0.9567 - val_loss: 0.9092 - val_accuracy: 0.8711

Epoch 00101: val_loss did not improve from 0.88250

Epoch 00101: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 102/2000
35/35 [==============================] - 16s 445ms/step - loss: 0.6400 - a

In [8]:
test_batch_size = 625
for fold in fold_list:
    nfold_model.evaluate_test(test_batch_size,fold)

1/1 [==============================] - 17s 17s/step - loss: 1.2165 - accuracy: 0.8784


In [9]:
predict = nfold_model.nfold_predict(test_batch_size,fold_list)
score_dict = nfold_model.nfold_score(preprocess.y_test,predict,average='macro')
print(score_dict)

{'accuracy': 0.8768, 'recall': 0.754438349973294, 'precision': 0.7915837236533958, 'f1': 0.7546620125736304}
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
